Runs an extended Kalman filter on Prof Flaxman's SEIIR predictions and
measurement data for New York State from mid-April to 7 July 2020.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

In [ ]:
%reset

In [1]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets_wDeaths
import seiir_compartmental

In [2]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]],
                      [10**-10]])

    beta_k = seiir['beta_pred'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']],
                                   [x_hat[5,0]]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2],
                          [0]])

    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0],
                         [0]])
    
    extra = np.array([[0],
                      [0],
                      [0],
                      [0],
                      [0],
                      [0]])

    # 6x6
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R, extra], axis=1)

    # 6x6
    # P_k1_prior = f_jacob * P * f_jacob^T + Q
    P_k1_prior = np.matmul(np.matmul(f_jacob, P), np.transpose(f_jacob)) + Q
    return P_k1_prior


def update_step(x_hat, x_hat_k1, P_k1, Rn, rho1, rho2, z_k):
    # 6x6
    ep = 10**-10
    H = np.array([[ep, 0, 0, 0, 0, 0],
                  [0, ep, 0, 0, 0, 0],
                  [0, 0, ep, rho1, 0, 0],
                  [0, 0, 0, rho2, 0, 0],
                  [0, 0, 0, 0, ep, 0],
                  [0, 0, 0, rho3, 0, ep]])
    #H = np.array([[0, 0, 0, 0, 0],
    #                  [0, 0, 0, 0, 0],
    #                  [0, 0, 0, rho1, 0],
    #                  [0, 0, 0, rho2, 0],
    #                  [0, 0, 0, 0, 0]])
    
    # Si = H * P_k1 * H^T + Rn
    Si = np.matmul(np.matmul(H, P_k1), np.transpose(H)) + Rn

    # K_new = P_k1 * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_k1, np.transpose(H)), np.linalg.inv(Si))
    y_new = np.matmul(H, x_hat)

    # 6x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_k1_post = P_k1 - np.matmul(np.matmul(K_new, Si), np.transpose(K_new))

    # joseph formulation
    #joe2 = np.matmul(np.matmul(K_new, Rn), np.transpose(K_new))
    #joe1 = np.eye(5) - np.matmul(K_new, H)
    #P_k1_post = np.matmul(np.matmul(joe1, P_k1), np.transpose(joe1)) - joe2

    return x_hat_k1_post, P_k1_post

def get_predict_measures():
    predictions = pd.read_csv(
        r'case_vs_symptom/kalman/predicted_measures/predictions.csv',
        header=None, names=['measure_date', 'prediction_date', 'prop', 'case',
                            'prop_pred7', 'case_pred7', 'prop_pred1',
                            'case_pred1'],
        index_col=False, usecols=[0, 4, 5])

    start_d = datetime.timedelta(days=int(predictions['measure_date'].iloc[0]))
    start = datetime.date(2019, 12, 31) + start_d
    end_d = datetime.timedelta(days=int(predictions['measure_date'].iloc[-1]))
    end = datetime.date(2019, 12, 31) + end_d

    measure_rng = pd.date_range(start=start, end=end)

    # code to save:
    # prediction_rng = pd.date_range(start=datetime.date(2019, 12, 31) +
    # datetime.timedelta(days=int(predictions['prediction_date'].iloc[0])),
    # end=datetime.date(2019, 12, 31) +
    # datetime.timedelta(days=int(predictions['prediction_date'].iloc[-1])))

    predictions.set_index(measure_rng, inplace=True)

    return predictions


def get_data_sets(state, fips=None):
    # the post hoc seiir model predictions provided by Prof Flaxman without
    # any kalman filtering:
    #seiir = pd.read_csv(r'data/seiir_compartments_post-hoc_ny_state_20200910.csv', header=0,
    #                    index_col='date', parse_dates=True)
    seiir = pd.read_csv(r'data/seiir_compartments_post-hoc_ny_state.csv', header=0,
                        index_col='date', parse_dates=True)
    

    fb_data = data_sets_wDeaths.create_symptom_df()
    fb_data_val = data_sets_wDeaths.create_symptom_df(valid=True)

    if fips is None:
        case_data = data_sets_wDeaths.create_case_df_state()
        case_data_geo = case_data.loc[state][['case_rate', 'death_rate']].copy()
        fb_data_geo = fb_data.loc[state].groupby('date').sum().copy()
        fb_data_val_geo = fb_data_val.loc[state].groupby('date').sum().copy()

    else:

        case_data = data_sets_wDeaths.create_case_df_county()
        case_data_geo = case_data.loc[fips][['case_rate', 'death_rate']].copy()

        if fips == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val.loc[(slice(None), '36081'), :].copy()
            fb_data_val_geo = fb_data_val.mean(level='date')
            for borough in nyc_fips:
                fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')
                fb_data_val_geo += fb_data_val.loc[(slice(None), borough), :].copy().mean(level='date')
        else:
            fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()
            fb_data_val_geo = fb_data_val.loc[(slice(None), fips), :].copy()
            # collapse down to a single index column (date)
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val_geo.mean(level='date')

    return seiir, fb_data_geo, fb_data_val_geo, case_data_geo


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()

In [3]:
# set constants
K0 = datetime.date(2020, 4, 12)

the_state = 'NY'
the_county = 'New York City'
#the_county = data_sets_wDeaths.get_fips('NY', 'Westchester')
constants = {
    'alpha': 0.948786,
    'gamma1': 0.500000,
    'gamma2': 0.662215,
    'sigma': 0.266635,
    'theta': 6.000000
    }

# set initial values for Kalman filter parameters
P_mult = 1
Q_mult = 1

# Rn is the R noise matrix; it remains constant thru the stepping of the
# Kalman filter
# prior to experiments, Rn_mult = 5*10**-4
Rn_mult = 5*10**-8

Rn_22 = 88
Rn_32 = 150

Rn_23 = 0
Rn_33 = 1

Rn_52 = 600

Rn = Rn_mult * np.array([[0, 0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0, 0],
                         [0, 0, Rn_22, Rn_23, 0, 0],
                         [0, 0, Rn_32, Rn_33, 0, 0],
                         [0, 0, 0, 0, 0, 0],
                         [0, 0, Rn_52, 0, 0, 0]])
Q = Q_mult * np.eye(6)
P = P_mult * np.eye(6)

# generate data
seiir, fb_data, fb_data_val, case_data = get_data_sets(
    data_sets_wDeaths.STATES[the_state], fips=the_county)

if the_county == 'New York City':
    county_pop = 0
    for each in ['36081', '36005', '36061', '36047', '36085']:
        this_count, state_pop = data_sets_wDeaths.get_pops(each)
        county_pop += this_count
else:
    county_pop, state_pop = data_sets_wDeaths.get_pops(the_county)

b = county_pop / state_pop

# calculate moving averages on the fb and case data
case_ma7 = case_data['case_rate'].rolling(window=7).mean()
case_ma7_all = case_ma7.iloc[6:]
case_data['death_rate'][case_data['death_rate'] < 0] = 0
deaths_ma28 = case_data['death_rate'].rolling(window=28).mean()
deaths_ma28_all = deaths_ma28.iloc[27:]

prop_ma7, n_ma7, num_stl_ma7 = calc_fb_ma7(fb_data)
prop_ma7_valid, n_ma7_valid, num_stl_ma7_valid = calc_fb_ma7(fb_data_val)


# get starting compartment values for the state level
x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)
# convert to county values:
x_hat_k0 = b * x_hat_state_k0
# set the death rate value in the county x vector
x_hat_k0[5,0] = deaths_ma28_all.loc[K0]
I2_county = x_hat_k0[3, 0]

rho1 = prop_ma7.loc['2020-04-12'] / I2_county
rho2 = case_ma7_all.loc['2020-04-12'] / I2_county
rho3 = 0.06 * rho2


# approximate rho values
# I2_county = b * I2
# prop_county = rho1 * I2_county
# case_county = rho2 * I2_county


# create empty dictionaries to hold the estimated values
prop_est = {}
case_est = {}
seiir_pred = {}
death_est = {}


x_death = deaths_ma28_all.loc[K0]

# Original data run ----------------
start = K0
d = start
while d <= datetime.date(2020, 9, 30):
    # each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)

    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    #x_hat_k[5, 0] = x_death
    x_hat_k1 = b * x_hat_state_k1
    #x_hat_k1[5, 0] = x_death

    # get measurements
    z_k = np.array([[0],
                    [0],
                    [prop_ma7.loc[d]],
                    [case_ma7_all.loc[d]],
                    [0],
                    [deaths_ma28_all.loc[d]]])

    # predict step
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)
    print('x_hat_post:')
    print(x_hat_post)
    
    # store the posterior death value (we don't have an modeled estimate of it)
    #x_death = x_hat_post[5, 0]

    # store estimated values for proportion and case rate
    indexDate = d + datetime.timedelta(days=7)
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    seiir_pred[indexDate] = b * x_hat_k1[3, 0]
    death_est[indexDate] = x_hat_post[5, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)
predicted_death = pd.Series(death_est)

step -----------
P:
[[ 1.10871079 -0.10871079  0.11656666  0.03781772 -0.15438439  0.        ]
 [-0.10871079  1.17980501 -0.18766089 -0.03781772  0.15438439  0.        ]
 [ 0.11656666 -0.18766089  1.32109422 -0.25        0.          0.        ]
 [ 0.03781772 -0.03781772 -0.25        1.68852871 -0.43852871  0.        ]
 [-0.15438439  0.15438439  0.         -0.43852871  1.43852871  0.        ]
 [ 0.          0.          0.          0.          0.          1.        ]]
P_post:
[[ 1.31215705e-10 -6.59776955e-11 -1.22791061e-09  7.95649962e-09
  -7.66643748e-10  1.73886271e-11]
 [-6.59776955e-11  7.39408534e-13  5.98939787e-11 -4.19609548e-10
   1.01506581e-11 -1.38756703e-12]
 [-4.27272967e-10 -3.99497102e-12  1.24181765e+00  1.34857628e-07
  -3.59025484e-11  2.98534145e-09]
 [ 2.93145424e-09 -1.86232141e-11  2.85575896e-10  1.13218102e-10
  -2.86348945e-10 -1.58142118e-08]
 [-7.66643693e-10  1.01507136e-11  5.69286896e-11 -8.68985994e-10
   1.35989886e-10 -2.01615116e-12]
 [ 2.62671598e-1

step -----------
P:
[[ 1.06174264e+00 -6.17426362e-02  1.36797536e-01 -1.36797516e-01
  -1.95608726e-08  0.00000000e+00]
 [-6.17426356e-02  1.06174264e+00 -1.36797534e-01  1.36797515e-01
   1.95673277e-08  0.00000000e+00]
 [ 1.36797556e-01 -1.36797556e-01  1.30308989e+00 -3.03089842e-01
  -4.33610399e-08  0.00000000e+00]
 [-1.36797564e-01  1.36797564e-01 -3.03089904e-01  1.30308986e+00
   4.33772325e-08  0.00000000e+00]
 [ 7.73654321e-09 -7.73845823e-09  1.71258159e-08 -1.71012532e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[-3.63367114e-11 -3.30542121e-11 -1.00009956e-08  6.71319278e-10
   5.79120397e-18  1.71775999e-12]
 [-3.30542330e-11  1.02445608e-10  1.04418042e-08  1.90780250e-09
   1.62119691e-17  4.91972233e-12]
 [-3.83659771e-11 -1.15376791e-10  1.21245979e+00  1.30767834e-07
  -5.31513174e-19  3.81238512e-09]
 [ 1.74738890e-10  4.86255092e-10 -5.25833213e-08 -1.32793776e

step -----------
P:
[[ 1.06140928e+00 -6.14092803e-02  1.36440811e-01 -1.36440791e-01
  -1.95030576e-08  0.00000000e+00]
 [-6.14092797e-02  1.06140928e+00 -1.36440809e-01  1.36440790e-01
   1.95249921e-08  0.00000000e+00]
 [ 1.36440831e-01 -1.36440831e-01  1.30314795e+00 -3.03147909e-01
  -4.33718605e-08  0.00000000e+00]
 [-1.36440839e-01  1.36440839e-01 -3.03147971e-01  1.30314793e+00
   4.33728857e-08  0.00000000e+00]
 [ 7.80580879e-09 -7.87277774e-09  1.73933559e-08 -1.73034272e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[-3.34359207e-11  2.63108008e-11 -1.00660947e-08  8.84011808e-10
   4.73372304e-18  2.47236129e-12]
 [ 2.63108008e-11 -1.91857641e-11  1.03869430e-08  2.32227423e-09
  -2.46861135e-18  8.38714754e-12]
 [-4.15805168e-11  2.57485699e-11  1.21259120e+00  1.30813728e-07
   1.19683108e-19  3.81435150e-09]
 [ 1.69363273e-10 -1.01306269e-10 -5.28755827e-08  2.41968667e

step -----------
P:
[[ 1.06209339e+00 -6.20933863e-02  1.37185886e-01 -1.37185867e-01
  -1.96633199e-08  0.00000000e+00]
 [-6.20933857e-02  1.06209339e+00 -1.37185885e-01  1.37185865e-01
   1.99671091e-08  0.00000000e+00]
 [ 1.37185906e-01 -1.37185906e-01  1.30309138e+00 -3.03091336e-01
  -4.37037126e-08  0.00000000e+00]
 [-1.37185915e-01  1.37185915e-01 -3.03091399e-01  1.30309136e+00
   4.33427167e-08  0.00000000e+00]
 [ 7.91954486e-09 -7.84282571e-09  1.74638813e-08 -1.75978071e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 2.26478614e-10 -1.12808075e-10 -9.84703311e-09 -5.98158745e-10
  -3.55153305e-17  1.66327958e-12]
 [-1.12808034e-10 -8.63309424e-13  1.10226857e-08 -4.01209621e-11
  -3.98194821e-19 -1.35667850e-13]
 [ 2.61327071e-10 -1.01877673e-11  1.21235620e+00  1.30856832e-07
  -2.94161969e-18  3.81139735e-09]
 [-1.07899731e-09 -7.34745598e-13 -5.33049786e-08  4.59092764e

step -----------
P:
[[ 1.06235790e+00 -6.23578990e-02  1.37475557e-01 -1.37475537e-01
  -1.96117429e-08  0.00000000e+00]
 [-6.23578982e-02  1.06235790e+00 -1.37475555e-01  1.37475536e-01
   1.90843702e-08  0.00000000e+00]
 [ 1.37475577e-01 -1.37475577e-01  1.30308160e+00 -3.03081552e-01
  -4.27180965e-08  0.00000000e+00]
 [-1.37475586e-01  1.37475585e-01 -3.03081614e-01  1.30308157e+00
   4.32572965e-08  0.00000000e+00]
 [ 7.93085081e-09 -7.87742410e-09  1.74221911e-08 -1.74637905e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 2.45564236e-10 -1.22569982e-10 -9.25344773e-09 -2.39595505e-09
  -3.84420344e-17 -4.28521765e-12]
 [-1.22569933e-10 -4.24327240e-13  1.08163275e-08  5.76111936e-11
  -2.58662686e-19  2.27832728e-13]
 [ 2.80914902e-10 -9.75922121e-12  1.21227212e+00  1.30649731e-07
  -2.79485800e-18  3.80924849e-09]
 [-1.15776538e-09 -8.05253086e-12 -5.22476252e-08  7.88511478e

...

step -----------
P:
[[ 1.09476724e+00 -9.47672447e-02  1.68722385e-01 -1.68722361e-01
  -2.41340987e-08  0.00000000e+00]
 [-9.47672435e-02  1.09476724e+00 -1.68722383e-01  1.68722359e-01
   2.38800913e-08  0.00000000e+00]
 [ 1.68722409e-01 -1.68722409e-01  1.30039121e+00 -3.00391166e-01
  -4.27164713e-08  0.00000000e+00]
 [-1.68722419e-01  1.68722419e-01 -3.00391227e-01  1.30039118e+00
   4.29738264e-08  0.00000000e+00]
 [ 8.81547098e-09 -8.78886782e-09  1.56658215e-08 -1.56890770e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 1.11989529e-10 -5.50255674e-11 -1.23667734e-08  7.75640108e-10
  -1.43865833e-17  4.15896433e-12]
 [-5.50255536e-11 -1.93867145e-12  1.34536293e-08 -5.21125088e-11
  -1.22356556e-19 -1.36011760e-13]
 [ 1.06390424e-10 -4.97252239e-12  1.20161488e+00  1.29814400e-07
  -1.92782714e-18  3.63768261e-09]
 [-4.26446295e-10 -1.25912891e-11 -4.69404810e-08 -3.64182018e

step -----------
P:
[[ 1.09259554e+00 -9.25955388e-02  1.66829185e-01 -1.66829161e-01
  -2.38952000e-08  0.00000000e+00]
 [-9.25955379e-02  1.09259554e+00 -1.66829183e-01  1.66829159e-01
   2.40341591e-08  0.00000000e+00]
 [ 1.66829209e-01 -1.66829209e-01  1.30057583e+00 -3.00575787e-01
  -4.31395407e-08  0.00000000e+00]
 [-1.66829218e-01  1.66829218e-01 -3.00575847e-01  1.30057580e+00
   4.29325450e-08  0.00000000e+00]
 [ 8.61495192e-09 -8.59050762e-09  1.55484879e-08 -1.56409688e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 1.24702471e-10 -6.32211644e-11 -1.22886034e-08  6.87287394e-10
  -1.63509520e-17  4.08109183e-12]
 [-6.32211367e-11  1.74016357e-12  1.33400819e-08  5.09991216e-11
   5.29220230e-19  1.32236284e-13]
 [ 1.18637794e-10 -7.86767873e-12  1.20230347e+00  1.29875939e-07
  -2.17139543e-18  3.64884661e-09]
 [-4.92295593e-10  1.29993238e-11 -4.75145898e-08  1.48505652e

step -----------
P:
[[ 1.09542765e+00 -9.54276485e-02  1.69312461e-01 -1.69312437e-01
  -2.42492329e-08  0.00000000e+00]
 [-9.54276475e-02  1.09542765e+00 -1.69312459e-01  1.69312435e-01
   2.42952034e-08  0.00000000e+00]
 [ 1.69312485e-01 -1.69312485e-01  1.30040260e+00 -3.00402554e-01
  -4.30120494e-08  0.00000000e+00]
 [-1.69312495e-01  1.69312495e-01 -3.00402615e-01  1.30040257e+00
   4.28890847e-08  0.00000000e+00]
 [ 8.77716536e-09 -8.75236011e-09  1.56062086e-08 -1.57080080e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 8.17657053e-12  4.08159617e-13 -1.28908191e-08  2.62077265e-09
  -1.37888030e-18  9.14550943e-12]
 [ 4.08145739e-13 -8.99413877e-12  1.35692853e-08  2.97128155e-11
  -7.95913185e-19  2.54396106e-13]
 [ 6.62833677e-12  8.60389537e-12  1.20144240e+00  1.30010364e-07
  -4.50396535e-19  3.63541395e-09]
 [-2.23524810e-11 -4.35862457e-11 -4.70974983e-08  1.65484071e

step -----------
P:
[[ 1.09860286e+00 -9.86028593e-02  1.72033060e-01 -1.72033035e-01
  -2.45768076e-08  0.00000000e+00]
 [-9.86028582e-02  1.09860286e+00 -1.72033058e-01  1.72033033e-01
   2.42758903e-08  0.00000000e+00]
 [ 1.72033084e-01 -1.72033084e-01  1.30014726e+00 -3.00147214e-01
  -4.25823078e-08  0.00000000e+00]
 [-1.72033094e-01  1.72033093e-01 -3.00147274e-01  1.30014723e+00
   4.28885239e-08  0.00000000e+00]
 [ 8.83353455e-09 -8.72563584e-09  1.52999751e-08 -1.54025751e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[-5.63307179e-11  2.54291171e-11 -1.25769000e-08  7.55992158e-10
   7.29829395e-18  1.88685376e-12]
 [ 2.54291310e-11  5.47295542e-12  1.36543360e-08 -5.87268845e-11
   4.05718856e-19 -2.82183431e-13]
 [-4.99845165e-11 -3.93085564e-12  1.20045189e+00  1.29680285e-07
   6.92793542e-19  3.61867216e-09]
 [ 2.10506834e-10  2.30384600e-11 -4.60074050e-08 -3.24882343e

step -----------
P:
[[ 1.10567504e+00 -1.05675043e-01  1.78003835e-01 -1.78003810e-01
  -2.54692495e-08  0.00000000e+00]
 [-1.05675041e-01  1.10567504e+00 -1.78003833e-01  1.78003808e-01
   2.54611842e-08  0.00000000e+00]
 [ 1.78003860e-01 -1.78003860e-01  1.29983778e+00 -2.99837733e-01
  -4.28453093e-08  0.00000000e+00]
 [-1.78003870e-01  1.78003870e-01 -2.99837793e-01  1.29983775e+00
   4.27895246e-08  0.00000000e+00]
 [ 8.97119983e-09 -8.96909313e-09  1.51576663e-08 -1.52236231e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[-1.96470173e-10  1.04863271e-10 -1.31637880e-08  4.78611706e-10
   2.28716002e-17 -8.40591521e-13]
 [ 1.04863243e-10 -1.32565070e-11  1.42853482e-08 -1.76525183e-11
  -8.50560045e-19  1.20915583e-13]
 [-1.80752413e-10  2.64448741e-11  1.19841196e+00  1.29429881e-07
   2.42201130e-18  3.58514499e-09]
 [ 7.21311899e-10 -5.23677768e-11 -4.52836968e-08 -9.40569844e

In [ ]:
deaths_ma28_all

In [6]:
# compute squared error for the overlapping time period -------------------------

left = predicted_seiir_prior.index[0]
right = d

seiir_err = case_ma7_all.loc[left:right] - predicted_seiir_prior[left:right]
seiir_sum_sq_err = (seiir_err**2).sum()
seiir_mse = seiir_sum_sq_err / seiir_err.count()

kalman_err = predicted_case.loc[left:right] - case_ma7_all.loc[left:right]
kalman_sum_sq_err = (kalman_err**2).sum()
kalman_mse = kalman_sum_sq_err / kalman_err.count()

deaths_err = predicted_death.loc[left:right] - deaths_ma28_all.loc[left:right]
deaths_sum_sq_err = (deaths_err**2).sum()
deaths_mse = deaths_sum_sq_err / deaths_err.count()


print('SSE between seiir forecast and case rate:', seiir_sum_sq_err)
print('MSE between seiir forecast and measured case rate:', seiir_mse)
print()
print('SSE between kalman forecast and measured case rate:', kalman_sum_sq_err)
print('MSE between kalman forecast and measured case rate:', kalman_mse)
print()
print('SSE between kalman deaths forecast and measured death rate:', deaths_sum_sq_err)
print('MSE between kalman deaths forecast and measured death rate:', deaths_mse)

SSE between seiir forecast and case rate: 1311332979.4650373
MSE between seiir forecast and measured case rate: 7899596.261837575

SSE between kalman forecast and measured case rate: 873046357.3456627
MSE between kalman forecast and measured case rate: 5356112.621752532

SSE between kalman deaths forecast and measured death rate: 3319120.753955934
MSE between kalman deaths forecast and measured death rate: 20362.704011999595


In [ ]:
# MASE calculation -----------------

left = predicted_case.index[0]
right = predicted_case.index[-1]

# take error between prediction and actual
e = (case_ma7_all.loc[left:right] - predicted_case.loc[left:right]).abs()

case_ma7_diff = case_ma7_all.diff(7)
denom = (case_ma7_diff.loc[left:right]).abs()

term = e.div(denom)
term.replace([np.inf, -np.inf], np.nan, inplace=True)
term.dropna(inplace=True)
mase = term.sum() / len(term)
print('mase:', mase)

In [ ]:
# Not currently used ---------------
# Validation data run ---------------------
start = prop_ma7_valid.index[0]
d = start
while d <= prop_ma7_valid.index[-1]:
    # each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)

    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    x_hat_k1 = b * x_hat_state_k1

    # get measurements
    z_k = np.array([[0],
                    [0],
                    [prop_ma7_valid.loc[d]],
                    [case_ma7_all.loc[d]],
                    [0]])

    # predict step
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)

    # store estimated values for proportion and case rate
    indexDate = d + datetime.timedelta(days=7)
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    seiir_pred[indexDate] = b * x_hat_k1[3, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)

In [4]:
# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

In [5]:
# Single plot with all lines

matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
tick_start = K0
tick_end = predicted_seiir_prior.index[-1]



fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.plot(predicted_death.index, predicted_death, label='Death Rate Forecast', c='blue', linewidth=width)
plt.plot(deaths_ma28_all.loc[start:d].index, deaths_ma28_all.loc[start:d], label='Death Rate', c='green', linewidth=width)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(prop_ma7.index, num_stl_ma7, c='red', label='Facebook Positive Symptoms',
         linewidth=width)
#plt.ylim(-.065, 2.5)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')

plt.show()

In [ ]:
fig4, ax41 = plt.subplots(1)
plt.plot(predicted_death.index, predicted_death, label='Death Rate Forecast', c='blue')
plt.plot(deaths_ma28_all.loc[start:d].index, deaths_ma28_all.loc[start:d], label='Death Rate', c='green')
plt.legend()
plt.show()

In [ ]:
# not currently used ---------------------
# plot validation data

fig_v, ax_v = plt.subplots(1)
plt.sca(ax_v)


plt.plot(case_ma7_all.loc[start:end].index, case_ma7_all.loc[start:end], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax_v.twinx()
plt.sca(ax32)
plt.plot(prop_ma7_valid.index, 100*prop_ma7_valid, c='red', label='Facebook Symptom Rate',
         linewidth=width)
plt.ylim(-.065, 2.5)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Percentage of Positive Symptom Response')
plt.legend(loc='upper right')

plt.show()

In [ ]:

# plot findings, multiple plots ------------------
matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
tick_start = K0
tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=tick_start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
width = 4
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
plt.ylabel('Number of Cases per Day')
# plt.xlabel('Date')
plt.legend(loc='upper left')

fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(prop_ma7.index, 100*prop_ma7, c='red', label='Facebook Symptom Rate',
         linewidth=width)
plt.ylim(-.065, 2.5)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Percentage of Positive Symptom Response')
plt.legend(loc='upper right')


plt.show()